In [1]:
from pathlib import Path
import numpy as np

In [2]:
lines = Path('../results/gen7vgc2018_ns_32_poke2vec.model').read_text().split('\n')
index2poke = []
n_vocab, n_units = map(int, lines[0].split())
vec = np.zeros((n_vocab, n_units))

for i, line in enumerate(lines[1:]):
    line = line.split()
    if len(line) == 0:
        break
    name, v = line[0], line[1:]
    v = list(map(float, v))
    index2poke.append(name)
    vec[i] = v

In [3]:
vec

array([[-0.19213457,  0.39025375, -0.36565906, ..., -0.18254557,
         0.10498768, -0.35236248],
       [-0.61358386, -0.14285038,  0.7308076 , ...,  1.9705235 ,
        -0.8620409 ,  1.1889096 ],
       [-0.36718768, -1.8430903 , -0.50245476, ..., -1.2942839 ,
         0.5975897 ,  1.1119113 ],
       ...,
       [-0.23748733,  0.18743554, -0.20721523, ...,  0.08759026,
        -0.22725153,  0.0684476 ],
       [ 0.02403239, -0.33811316, -0.06650057, ..., -0.12179834,
         0.23604423,  0.5437267 ],
       [-0.06221102,  0.0660575 , -1.3806828 , ...,  0.20800947,
        -0.54373425,  0.40150735]])

In [4]:
from gensim.models.keyedvectors import Vocab
vocab = {x: Vocab(index=i) for i, x in enumerate(index2poke)}

In [5]:
from gensim.models import KeyedVectors
poke2vec = KeyedVectors(vec.shape[1])

In [6]:
poke2vec.vectors = vec
poke2vec.vocab = vocab
poke2vec.index2entity = index2poke

In [7]:
import pandas as pd
df = pd.read_csv('../pokemon.csv')
df.head()

,Unnamed: 0,Japanese,English,French,Spanish,German,Italian,Korean,MandarinChinese,CantoneseChinese
0,0,フシギダネ,Bulbasaur,Bulbizarre,Bulbasaur,Bisasam,Bulbasaur,이상해씨,妙蛙种子,妙蛙種子
1,1,フシギソウ,Ivysaur,Herbizarre,Ivysaur,Bisaknosp,Ivysaur,이상해풀,妙蛙草,妙蛙草
2,2,フシギバナ,Venusaur,Florizarre,Venusaur,Bisaflor,Venusaur,이상해꽃,妙蛙花,妙蛙花
3,3,ヒトカゲ,Charmander,Salamèche,Charmander,Glumanda,Charmander,파이리,小火龙,小火龍
4,4,リザード,Charmeleon,Reptincel,Charmeleon,Glutexo,Charmeleon,리자드,火恐龙,火恐龍


In [8]:
import re
def to_id(name):
    return re.sub(r'[^a-z0-9]+', '', name.lower())

In [9]:
_to_ja = {to_id(y): x for x, y in zip(df.Japanese,df.English)}
def to_ja(name):
    if name in _to_ja:
        return _to_ja[name]
    if name.startswith('vivillon'):
        return _to_ja['vivillon']
    if name.startswith('meowstic'):
        return 'ニャオニクス' + ' ♂' if name[-1] == 'm' else '♀'
    return name

In [10]:
ret = poke2vec.most_similar(positive=['zapdos'])
for name, sim in ret:
    print(to_ja(name), sim)

thundurustherian 0.8745791912078857
ライチュウ 0.7674826383590698
naganadel 0.7397462129592896
thundurus 0.7042409777641296
rotomheat 0.6822828054428101
フワライド 0.6556263566017151
カプ・コケコ 0.5665922164916992
トゲデマル 0.5359745025634766
ジャローダ 0.4981088638305664
デンジュモク 0.4966435134410858


In [11]:
ret = poke2vec.most_similar(positive=['thundurustherian', 'zapdos'], negative=['manectric'])
for name, sim in ret:
    print(to_ja(name), sim)

ライチュウ 0.7268299460411072
naganadel 0.6770119071006775
thundurus 0.672311007976532
rotomheat 0.6123127937316895
フワライド 0.5884656310081482
ジャローダ 0.580478310585022
rotomwash 0.5405862927436829
カプ・コケコ 0.5096319317817688
トゲデマル 0.5073633790016174
トゲキッス 0.48859071731567383
